In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'memotion_analysis/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Libraries

In [0]:
!pip install transformers

In [0]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [0]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image as keras_image
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, concatenate
from transformers import TFBertModel
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

### DATA Loading

In [0]:
dataframe = pd.read_csv("/content/train_final_V2.csv")

In [0]:
dataframe.dropna(axis = 0,inplace = True)
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7614 entries, 0 to 7622
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               7614 non-null   object
 1   filepath           7614 non-null   object
 2   Overall_Sentiment  7614 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 237.9+ KB


In [0]:
# Split train and validation dataframes
train_test_factor = 0.85
mask = np.random.rand(len(dataframe)) < train_test_factor
validation_dataframe = dataframe[~mask]
train_dataframe = dataframe[mask]

# Shuffle train dataframe
train_dataframe = train_dataframe.reset_index(drop=True)
print(train_dataframe['Overall_Sentiment'].value_counts())
validation_dataframe = validation_dataframe.reset_index(drop=True)
print(validation_dataframe['Overall_Sentiment'].value_counts())

2    2878
0    1914
1    1639
Name: Overall_Sentiment, dtype: int64
2    525
0    351
1    307
Name: Overall_Sentiment, dtype: int64


In [0]:
train = {'image':[],'text':[],'label':[]}
validation = {'image':[],'text':[],'label':[]}

In [0]:
def preprocess_image(image):
  img = image.copy()
  img = keras_image.img_to_array(img)
  img = img//255.0
  return img

In [0]:
# train
for index,filepath in enumerate(train_dataframe['filepath']):
  try:
    image = keras_image.load_img(filepath, target_size=(224, 224),interpolation='bicubic')
  except Exception as e:
    print(e)
    continue

  image = preprocess_image(image)
  train['image'].append(image)
  train['text'].append(train_dataframe.text[index])
  train['label'].append(train_dataframe.Overall_Sentiment[index])

# validation
for index,filepath in enumerate(validation_dataframe['filepath']):
  try:
    image = keras_image.load_img(filepath, target_size=(224, 224),interpolation='bicubic')
  except Exception as e:
    print(e)
    continue

  image = preprocess_image(image)
  validation['image'].append(image)
  validation['text'].append(validation_dataframe.text[index])
  validation['label'].append(validation_dataframe.Overall_Sentiment[index])

[Errno 2] No such file or directory: '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/dr_evil_football-meme-009-dr-evil-super-bowl.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/harvey_Steve-Harvey-Miss-Universe-Screw-Up-Memes12.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/hitler_funny-memes-on-Hitler-2.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/harvey_14.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/sexist_meme4_640x700.jpg'


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/neutral/seal_seal-cute-W630.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/negative/hitler_Funny-memes-of-hitler.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/minion_Minion-Memes191.jpg'
image file is truncated (52 bytes not processed)
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/neutral/harry_harry-potter-memes-600-pokemon.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/neutral/barney_Barneys-Dating-Advice-e1333644417653.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/neutral/spiderman_60s-spiderman-meme-collection-1mut.com-8.png'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/neutral/skeptical_skeptical-baby-meme-11.jpg'
cannot identify image file

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


image file is truncated (1 bytes not processed)
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/positive/harry_serveous-snape-memes.jpg'
cannot identify image file '/content/gdrive/My Drive/memotion_analysis/dataset/train/neutral/avengers_Avengers-Endgame-Meme-003-first-watched-cursed-with-knowledge.jpg'


In [0]:
np.save("/content/gdrive/My Drive/memotion_analysis/train.npy",train['image'])
np.save("/content/gdrive/My Drive/memotion_analysis/val.npy",validation['image'])

In [0]:
(len(train['image']),len(train['text']),len(train['label']))

(6409, 6409, 6409)

In [0]:
(len(validation['image']),len(validation['text']),len(validation['label']))

(1180, 1180, 1180)

In [0]:
train = pd.DataFrame(train)
val = pd.DataFrame(validation)

In [0]:
train.to_csv("/content/gdrive/My Drive/memotion_analysis/train.csv")

In [0]:
val.to_csv("/content/gdrive/My Drive/memotion_analysis/val.csv")

## Main Data Loading

In [0]:
train = pd.read_csv("/content/gdrive/My Drive/memotion_analysis/train.csv")
val = pd.read_csv("/content/gdrive/My Drive/memotion_analysis/val.csv")

## Image data

In [0]:
train_images = np.load('/content/gdrive/My Drive/memotion_analysis/train.npy')
test_images = np.load('/content/gdrive/My Drive/memotion_analysis/val.npy')

## Text data processing

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
input_id_mask_train = [tokenizer.encode_plus(x, max_length=52, pad_to_max_length=True,) for x in train['text']]
input_id_mask_val = [tokenizer.encode_plus(x, max_length=52, pad_to_max_length=True) for x in val['text']]

In [0]:
train_text = pd.DataFrame(input_id_mask_train)
val_text = pd.DataFrame(input_id_mask_val)

In [0]:
train_text.columns

Index(['input_ids', 'token_type_ids', 'attention_mask'], dtype='object')

## Processed input for model

In [0]:
train_input_ids = tf.stack(train_text.input_ids.apply(tf.convert_to_tensor).values)
train_input_masks = tf.stack(train_text.attention_mask.apply(tf.convert_to_tensor).values)
train_segment_ids = tf.stack(train_text.token_type_ids.apply(tf.convert_to_tensor).values)

test_input_ids = tf.stack(val_text.input_ids.apply(tf.convert_to_tensor).values)
test_input_masks = tf.stack(val_text.attention_mask.apply(tf.convert_to_tensor).values)
test_segment_ids = tf.stack(val_text.token_type_ids.apply(tf.convert_to_tensor).values)

In [0]:
train_imagesX = tf.convert_to_tensor(train_images)
trainY = tf.stack(train.label.values)

test_imagesX = tf.convert_to_tensor(test_images)
testY = tf.stack(val.label.values)

In [0]:
train_input_ids.shape

TensorShape([50, 52])

In [0]:
test_input_ids.shape

TensorShape([10, 52])

In [0]:
train_imagesX.shape

TensorShape([50, 224, 224, 3])

In [0]:
trainY.shape

TensorShape([50])

## Analysis

In [0]:
max([len(s.split()) for s in train['text']])

158

In [0]:
l=[len(s.split()) for s in train['text']]
count=0.0
for i in l:
    if i>52:
        count+=1
print(count,len(l))
print(count/len(l))
MAX_SEQUENCE_LENGTH = 52 ## State after running this cell

13.0 6409
0.002028397565922921


## Multi-Modal

In [0]:
bert_model = TFBertModel.from_pretrained("bert-base-uncased")
vis_model = VGG19(include_top=False)

token_inputs = Input((MAX_SEQUENCE_LENGTH), dtype=tf.int32, name='input_word_ids')
mask_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
seg_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')
bert_output = bert_model([token_inputs, mask_inputs, seg_inputs])[1]
bert_output = Dense(768,activation='relu')(bert_output)
bert_output = Dropout(0.4)(bert_output)
text_repr = Dense(32, activation='relu')(bert_output)

vis_input = Input(shape=(224,224,3))
vis_feature = vis_model(vis_input)
flat = Flatten()(vis_feature)
flat = Dense(2742, activation='relu')(flat)
flat = Dropout(0.4)(flat)
visual_repr = Dense(32,activation='relu')(flat)

combine_repr = concatenate([text_repr, visual_repr])
com_drop= Dropout(0.4)(combine_repr)
prediction = Dense(1,activation='sigmoid')(com_drop)

In [0]:
memsem = Model(inputs = [token_inputs, mask_inputs, seg_inputs,vis_input], outputs = prediction)
memsem.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [0]:
plot_model(
    memsem, to_file='model.png', show_shapes=False, show_layer_names=True,
    rankdir='TB', expand_nested=True, dpi=96
)

In [0]:
checkpoint = ModelCheckpoint('model-{epoch:03d}-{val_acc:03f}.h5', verbose=1, monitor='val_acc',save_best_only=True, mode='max')  

In [0]:
out = memsem.fit([train_input_ids,
                  train_input_masks,
                  train_segment_ids,
                  train_imagesX],
                  trainY,
                    batch_size=50,
                    epochs=20,
                    
                    shuffle=True,
                    validation_data=([test_input_ids,
                                      test_input_masks, 
                                      test_segment_ids,
                                      test_imagesX],
                                     testY))